In [1]:
import pandas as pd
import ast
import pycrfsuite

In [27]:
# Training
training = pd.read_csv('./data/training.csv').drop('Unnamed: 0', axis=1)
training['features'] = training['features'].apply(lambda x: ast.literal_eval(x))
training['y'] = training['y'].apply(lambda x: ast.literal_eval(x))
# training.head()

# Testing
testing = pd.read_csv('./data/testing.csv').drop('Unnamed: 0', axis=1)
testing['features'] = testing['features'].apply(lambda x: ast.literal_eval(x))
testing['y'] = testing['y'].apply(lambda x: ast.literal_eval(x))
testing.head()

,features,y
0,"[[14, M, 0.015490478847519541, 0, 1], [giờ, Nu...","[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, ..."
1,"[[Theo, V, 0.0, 0, 0], [đó, P, 0.0563213809701...","[1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, ..."
2,"[[Ông, Nc, 0.0, 1, 0], [Danny_Le, Np, 0.0, 1, ...","[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, ..."
3,"[[Hợp_đồng, N, 0.0, 0, 0], [vay, V, 0.28663658...","[1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, ..."
4,"[[Theo, V, 0.0, 0, 0], [đó, P, 0.0931123754746...","[1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, ..."


In [28]:
training_data = training['features'].tolist()
testing_data = testing['features'].tolist()

In [50]:
def convert(y):
    y_ = y.copy()
    for i in range(len(y)):
        if y_[i] == 0:
            y_[i] = 'N'
        else:
            y_[i] = 'I'
    return y_

training['y_convert'] = training['y'].apply(lambda x: convert(x))
testing['y_convert'] = testing['y'].apply(lambda x: convert(x))

In [39]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    tfidf = doc[i][2]
    ner = doc[i][3]
    title = doc[i][4]
    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isner=%s' % ner,
        'tfidf=%f' % tfidf,
        'word.istitle=%s' % title,
        'postag=' + postag
    ]

    # Features for words that are not at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        tfidf1 = doc[i-1][2]
        ner1 = doc[i-1][3]
        title1 = doc[i-1][4]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.isner=%s' % ner1,
            '-1:tfidf=%f' % tfidf1,
            '-1:word.istitle=%s' % title1,
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        tfidf1 = doc[i+1][2]
        ner1 = doc[i+1][3]
        title1 = doc[i+1][4]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.isner=%s' % ner1,
            '+1:tfidf=%f' % tfidf1,
            '+1:word.istitle=%s' % title1,
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [51]:
# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

X_train = [extract_features(doc) for doc in training_data]
X_test = [extract_features(doc) for doc in testing_data]
y_train = training['y_convert'].tolist()
y_test = testing['y_convert'].tolist()

In [52]:
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('./model/crf.model')

conds required for this iteration: 0.102

***** Iteration #106 *****
Loss: 2154.737489
Feature norm: 204.847360
Error norm: 4.257674
Active features: 9834
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.145

***** Iteration #107 *****
Loss: 2154.711909
Feature norm: 204.852737
Error norm: 2.354708
Active features: 9833
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.146

***** Iteration #108 *****
Loss: 2154.689544
Feature norm: 204.858816
Error norm: 7.650414
Active features: 9831
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.143

***** Iteration #109 *****
Loss: 2154.662047
Feature norm: 204.863607
Error norm: 6.365033
Active features: 9823
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.146

***** Iteration #110 *****
Loss: 2154.639199
Feature norm: 204.868864
Error norm: 7.962409
Active features: 9817
Line search trials: 1

In [53]:
tagger = pycrfsuite.Tagger()
tagger.open('./model/crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

theo (I)
đó (N)
hội_đồng_quản_trị (I)
thông_qua (I)
phương_án (I)
phân_phối (I)
lợi_nhuận (I)
năm (I)
2015 (N)
là (I)
10% (N)
trong (N)
đó (N)
đã (N)
thực_hiện (I)
chi_trả (N)
đợt (I)
một (I)
10% (N)
dự_kiến (I)
quý (I)
ii (N)
/ (N)
2016 (N)
sẽ (I)
thực_hiện (I)
10% (N)
còn (N)
lại (N)
ngoài_ra (I)
từ (N)
nguồn (I)
lợi_nhuận (I)
được (I)
giữ (I)
lại (N)
fpt (I)
đưa (N)
ra (N)
phương (N)
chi_trả (N)
cổ_tức (I)
bằng (I)
cổ_phiếu (I)
theo (I)
tỷ_lệ (I)
là (I)
15% (N)
và (N)
sau (N)
khi (N)
được (I)
đhđcđ (N)
phê_duyệt (N)
fpt (I)
sẽ (I)
thực_hiện (I)
ngoài_ra (I)
hội_đồng_quản_trị (I)
cũng (N)
đề_xuất (I)
chính_sách (I)
trả (I)
cổ_tức (I)
bằng (I)
tiền_mặt (I)
mặt (N)
trong (N)
2016 (N)
với (N)
tỷ_lệ (I)
là (I)
20% (N)
tải (I)
file (N)
gốc (N)


In [56]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"N": 0, "I": 1}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["N", "I"]))

              precision    recall  f1-score   support

           N       0.99      0.99      0.99     12928
           I       1.00      0.99      0.99     16625

    accuracy                           0.99     29553
   macro avg       0.99      0.99      0.99     29553
weighted avg       0.99      0.99      0.99     29553



In [57]:
y_train_pred = [tagger.tag(xseq) for xseq in X_train]

predictions = np.array([labels[tag] for row in y_train_pred for tag in row])
truths = np.array([labels[tag] for row in y_train for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["N", "I"]))

              precision    recall  f1-score   support

           N       1.00      1.00      1.00     59599
           I       1.00      1.00      1.00     76329

    accuracy                           1.00    135928
   macro avg       1.00      1.00      1.00    135928
weighted avg       1.00      1.00      1.00    135928

